In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!cd /home/sangwon/.local/lib/python3.10/site-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar

--2024-06-20 02:22:51--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.14/redshift-jdbc42-2.1.0.14.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.221.168, 52.216.58.24, 16.182.74.112, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.221.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1048241 (1024K) [binary/octet-stream]
Saving to: ‘redshift-jdbc42-2.1.0.14.jar.3’

redshift-jdbc42-2.1 100%[===================>]   1024K   833KB/s    in 1.2s    

2024-06-20 02:22:53 (833 KB/s) - ‘redshift-jdbc42-2.1.0.14.jar.3’ saved [1048241/1048241]



In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("PySpark DataFrame #5") \
    .config("spark.jars", "/home/sangwon/.local/lib/python3.10/site-packages/pyspark/jars/redshift-jdbc42-2.1.0.14.jar") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/06/20 02:23:01 WARN Utils: Your hostname, LAPTOP-8CK94L0D resolves to a loopback address: 127.0.1.1; using 172.21.9.191 instead (on interface eth0)
24/06/20 02:23:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/06/20 02:23:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/20 02:23:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Redshift와 연결해서 테이블들을 데이터프레임으로 로딩하기

In [4]:
df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234") \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

In [5]:
df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.jdbc42.Driver") \
    .option("url", "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=guest&password=Guest1234") \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

In [6]:
df_user_session_channel.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)



In [7]:
df_user_session_channel.rdd.getNumPartitions()

1

In [8]:
df_session_timestamp.printSchema()

root
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [9]:
df_session_timestamp.rdd.getNumPartitions()

1

## DataFrame으로 처리하기

In [10]:
join_expr = df_user_session_channel.sessionid == df_session_timestamp.sessionid
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner")

In [11]:
session_df.printSchema()

root
 |-- userid: integer (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- channel: string (nullable = true)
 |-- sessionid: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [12]:
session_df.show(5)

+------+--------------------+--------+--------------------+--------------------+
|userid|           sessionid| channel|           sessionid|                  ts|
+------+--------------------+--------+--------------------+--------------------+
|  1501|0135456d6a3c1051f...|  Google|0135456d6a3c1051f...|2019-09-24 14:49:...|
|   876|01a416a7e28d0d229...|Facebook|01a416a7e28d0d229...|2019-05-26 14:23:...|
|   243|0226aa5193c66d990...|  Google|0226aa5193c66d990...|2019-07-01 23:04:...|
|  2133|02ea66ee24d57e285...| Organic|02ea66ee24d57e285...| 2019-10-28 22:50:40|
|  1961|0302915889fa38fe5...| Youtube|0302915889fa38fe5...| 2019-11-29 15:16:49|
+------+--------------------+--------+--------------------+--------------------+
only showing top 5 rows



In [17]:
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner").select(
    "userid", "sessionid", "channel", "ts"
)

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `sessionid` is ambiguous, could be: [`sessionid`, `sessionid`].

In [14]:
session_df = df_user_session_channel.join(df_session_timestamp, join_expr, "inner").select(
    "userid", df_user_session_channel.sessionid, "channel", "ts"
)

In [15]:
channel_count_df = session_df.groupby("channel").count().orderBy("count", ascending=False)

In [16]:
channel_count_df.show()

+---------+-----+
|  channel|count|
+---------+-----+
|  Youtube|17091|
|   Google|16982|
|    Naver|16921|
|  Organic|16904|
|Instagram|16831|
| Facebook|16791|
+---------+-----+



In [18]:
from pyspark.sql.functions import date_format, asc, countDistinct

session_df.withColumn('month', date_format('ts', 'yyyy-MM')).groupby('month').\
    agg(countDistinct("userid").alias("mau")).sort(asc('month')).show()

+-------+---+
|  month|mau|
+-------+---+
|2019-05|281|
|2019-06|459|
|2019-07|623|
|2019-08|662|
|2019-09|639|
|2019-10|763|
|2019-11|721|
+-------+---+



## Spark SQL로  처리하기

In [19]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")

In [20]:
df_session_timestamp.createOrReplaceTempView("session_timestamp")

In [21]:
channel_count_df = spark.sql("""
    SELECT channel, count(distinct userId) uniqueUsers
    FROM session_timestamp st
    JOIN user_session_channel usc ON st.sessionID = usc.sessionID
    GROUP BY 1
    ORDER BY 1
""")

In [22]:
channel_count_df

DataFrame[channel: string, uniqueUsers: bigint]

In [23]:
channel_count_df.show()

+---------+-----------+
|  channel|uniqueUsers|
+---------+-----------+
| Facebook|        889|
|   Google|        893|
|Instagram|        895|
|    Naver|        882|
|  Organic|        895|
|  Youtube|        889|
+---------+-----------+



In [24]:
mau_df = spark.sql("""
SELECT 
  LEFT(A.ts, 7) AS month,
  COUNT(DISTINCT B.userid) AS mau
FROM session_timestamp A
JOIN user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1      
ORDER BY 1 DESC""")

In [25]:
mau_df.collect()

[Row(month='2019-11', mau=721),
 Row(month='2019-10', mau=763),
 Row(month='2019-09', mau=639),
 Row(month='2019-08', mau=662),
 Row(month='2019-07', mau=623),
 Row(month='2019-06', mau=459),
 Row(month='2019-05', mau=281)]